In [1]:
#################################################################
# File       : Read_and_Display_OMETIFF_CZI.ipynb
# Version    : 0.1
# Author     : czsrh
# Date       : 25.10.2019
# Insitution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own Risk.
# Feedback or Improvements are welcome.
##################################################################

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from apeer_ometiff_library import io, processing, omexmlClass
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf

In [9]:
# define your testfiles here

imgdict = {
    1:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.czi',
    2:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff',
    3:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small_Fiji.ome.tiff',
    4:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=15_Z=20_CH=2_DCV.czi'
}

filename = imgdict[3]

In [10]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    print('Array Shape: ', array.shape)
    print('Sizes BF: ', metadata['Sizes BF'])
    print(metadata['DimOrder BF'])
    print(metadata['DimOrder BF Array'])
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata = imf.get_array_czi(filename, replacezero=False)
    print('Array Shape: ', array.shape)
    print(metadata['Shape'])
    print(metadata['Axes'])

Image Type:  ometiff
Getting OME-TIFF Metadata ...
Array Shape:  (10, 15, 2, 256, 256)
Sizes BF:  [1, 10, 15, 2, 256, 256]
XYCZT
TZCYX


In [11]:
show_ipyviewer = False

if show_ipyviewer:

    #out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
    if metadata['Extension'] == 'ome.tiff':
        ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
    if metadata['Extension'] == 'czi':
        ui, out = imf.create_ipyviewer_czi(array, metadata)

    display(ui, out)

In [12]:
import time

# switch to qt5 backend for napari viewer and wait a few seconds
%gui qt5
time.sleep(5)

In [13]:
napari_auto = True

# try to configre napari automatiaclly based on metadata
if napari_auto:
    imf.show_napari(array, metadata)

In [ ]:
# configure napari viewer manually - check array diemsnion and dimension order carefully 
if not napari_auto:
    
    # get the scalefactors
    scalefactors = imf.get_scalefactor(metadata)
    print(scalefactors)
    
    viewer = napari.Viewer()
    # add every channel as a single layer
    for ch in range(metadata['SizeC']):
        chname = metadata['Channels'][ch]
        viewer.add_image(array[ch, :, :, :, :], name=chname, scale=scalefactors['zx'])